In [2]:
import time
import re
import pandas as pd
import csv

import requests
from bs4 import BeautifulSoup as bs
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.660 YaBrowser/23.9.0.0 Safari/537.36',
    'Host': 'chelyabinsk.hh.ru',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'ru,en-US;q=0.7,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'DNT': '1',
    'Connection': 'keep-alive'
}
sess = requests.Session()
sess.headers.update(headers)



In [4]:
driver = uc.Chrome(headless=True,use_subprocess=False)

In [13]:
# html = driver.page_source

## Get URLs

In [5]:
def get_urls(start_html, num_of_vacancy):

    start_urls = list(set(re.findall('http[s]?://chelyabinsk.hh.ru/vacancy/[0-9]+', start_html)))
    
    while len(start_urls) < num_of_vacancy:
    
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        button_next = driver.find_element(By.LINK_TEXT, 'дальше')
        button_next.click()
        time.sleep(3)
        html = driver.page_source
        current_urls = re.findall('http[s]?://chelyabinsk.hh.ru/vacancy/[0-9]+', html)
        for url in current_urls:
            start_urls.append(url)
        print('Get unique URLS: ', len(set(current_urls)))
        
    all_urls = list(set(start_urls))

    return all_urls

In [6]:
location_codes = []

In [7]:
# получаю коды локаций в которых больше 2к резюме

for i in range(1,113):
    url = f"https://chelyabinsk.hh.ru/search/vacancy?L_save_area=true&items_on_page=100&area={i}&search_field=name&search_field=company_name&search_field=description&text=&enable_snippets=false"
    response = sess.get(url)
    page = bs(response.text, 'html.parser')
    find_result = page.find('h1', class_="bloko-header-section-3")
    try:
        num_of_vacancy = find_result.text[8:-9]
        num_of_vacancy = re.sub(r'\D', '', num_of_vacancy)
        if num_of_vacancy:
            if int(num_of_vacancy) > 2000:
                location_codes.append(i)
                print(num_of_vacancy)
    except:
        pass

218379
92909
37515
29588
9627
2763
3173
3695
30750
5111
4023
3483
8166
5798
9644
3636
15944
5791
12320
39285
5836
4436
7975
2843
6237
3665
26320
20272
3423
3978
5833
2332
2503
3378
2780
13737
3091
5475
5777
14692
2078
2107
19146
7251
18522
9066
2719
3267
4751
3018
21438
7130
8263
7850
14034
8664
5800
17624
9223
21658
3632
5717
8614


In [8]:
all_urls = []

In [9]:
for i in location_codes:
    
    try:
        url = f"https://chelyabinsk.hh.ru/search/vacancy?L_save_area=true&items_on_page=100&area={i}&search_field=name&search_field=company_name&search_field=description&text=&enable_snippets=false"
        driver.get(url)
        html = driver.page_source
        urls = get_urls(html, 3900)
        all_urls.append(urls)
    except:
        pass
        

Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  200
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  201
Get unique URLS:  202
Get unique URLS:  201
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  201
Get unique URLS:  202
Get unique URLS:  200
Get unique URLS:  202
Get unique URLS:  202
Get unique URLS:  201
Get unique URLS:  201
Get unique

# i stoped here

In [12]:
all_urls

[['https://chelyabinsk.hh.ru/vacancy/89969928',
  'https://chelyabinsk.hh.ru/vacancy/89242150',
  'https://chelyabinsk.hh.ru/vacancy/90728179',
  'https://chelyabinsk.hh.ru/vacancy/90154888',
  'https://chelyabinsk.hh.ru/vacancy/83776608',
  'https://chelyabinsk.hh.ru/vacancy/83138835',
  'https://chelyabinsk.hh.ru/vacancy/90869529',
  'https://chelyabinsk.hh.ru/vacancy/90060992',
  'https://chelyabinsk.hh.ru/vacancy/90854894',
  'https://chelyabinsk.hh.ru/vacancy/90825767',
  'https://chelyabinsk.hh.ru/vacancy/90092683',
  'https://chelyabinsk.hh.ru/vacancy/90934361',
  'https://chelyabinsk.hh.ru/vacancy/84953611',
  'https://chelyabinsk.hh.ru/vacancy/89191107',
  'https://chelyabinsk.hh.ru/vacancy/90160309',
  'https://chelyabinsk.hh.ru/vacancy/90002089',
  'https://chelyabinsk.hh.ru/vacancy/84566694',
  'https://chelyabinsk.hh.ru/vacancy/87537738',
  'https://chelyabinsk.hh.ru/vacancy/90790169',
  'https://chelyabinsk.hh.ru/vacancy/90073765',
  'https://chelyabinsk.hh.ru/vacancy/908

In [13]:
all_urls_1d = []

In [14]:
for i in all_urls:
    for j in i:
        all_urls_1d.append(j)

In [15]:
all_urls = list(set(all_urls_1d))

In [16]:
len(all_urls)

29940

In [17]:
urls_series = pd.Series(all_urls)
urls_series.to_csv('data/hh_vacancies_urls.csv', index=False)

## Get data

In [19]:
with open('data/hh_vacancies_urls.csv', newline='') as f:
    reader = csv.reader(f)
    all_urls = list(reader)

In [20]:
len(all_urls)

29941

In [22]:
df = pd.DataFrame({
    'id':[],
    'title':[],
    'description':[],
    'salary':[],
    'currency':[],
    'compensation_type':[]
})

In [24]:
for n, url in enumerate(all_urls_1d):
    
    response = sess.get(url)
    page = bs(response.text, 'html.parser')
    id = url[-8:]
    dirty_description = page.find('div', class_="g-user-content")
    description = re.sub(r'(\<(/?[^>]+)>)', '', str(dirty_description))
    dirty_title = page.find('h1', class_="bloko-header-section-1")
    title = re.sub(r'(\<(/?[^>]+)>)', '', str(dirty_title))
    dirty_salary = page.find('span', class_="bloko-header-section-2 bloko-header-section-2_lite")
    salary = re.sub(r'(\<(/?[^>]+)>)', '', str(dirty_salary))
    df.loc[n, 'id'] = id
    df.loc[n, 'title'] = title
    df.loc[n, 'description'] = description
    df.loc[n, 'salary'] = salary
    
    if n % 2000 == 0:
        df.to_csv(f'data\my_data_hh_{n}.csv', index=False)
    
    print(n)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [33]:
df

,id,title,description,salary,currency,compensation_type
0,87781859,Операционный менеджер,"Федеральная мебельная компания ""Агат"" является...",ООО Федеральная мебельная компания Агат,NaN,NaN
1,89380258,"Менеджер по работе с клиентами, удаленно",Мы - Благотворительный Фонд «Жизнь Одна». Помо...,от 60 000 до 100 000 ₽ до вычета налогов,NaN,NaN
2,88208555,Начинающий специалист по заполнению заявок (уд...,Консультировать клиентов по вопросам сервиса. ...,от 40 000 ₽ до вычета налогов,NaN,NaN
3,89153564,Начальник смены,Обязанности: Ведение учета продукции и предос...,от 50 000 ₽ на руки,NaN,NaN
4,89677308,Менеджер по развитию розничной сети,Компания ДОБРОЦЕН - это крупнейшая федеральная...,до 50 000 ₽ на руки,NaN,NaN
...,...,...,...,...,...,...
11985,89463605,Фельдшер-лаборант клинической лаборатории,Обязанности: Контролировать правильность дост...,от 35 000 ₽ до вычета налогов,NaN,NaN
11986,89187126,Мастер смены,На пищевое производство требуется мастер смены...,от 65 000 ₽ на руки,NaN,NaN
11987,87605248,Бетонщик-арматурщик,Обязанности: Бетонные и арматурные работы Тр...,от 95 000 до 97 000 ₽ на руки,NaN,NaN
11988,89837753,Товаровед,ЮВЕЛИРНАЯ СЕТЬ 585 GOLD ПРИГЛАШАЕТ ТОВАРОВЕДА ...,от 40 000 ₽ на руки,NaN,NaN
